# Self-Driving Car Engineer Nanodegree


## Project: **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

In addition to implementing code, there is a brief writeup to complete. The writeup should be completed in a separate file, which can be either a markdown file or a pdf document. There is a [write up template](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) that can be used to guide the writing process. Completing both the code in the Ipython notebook and the writeup template will cover all of the [rubric points](https://review.udacity.com/#!/rubrics/322/view) for this project.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note: If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".**

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="examples/line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="examples/laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

**Run the cell below to import some packages.  If you get an `import error` for a package you've already installed, try changing your kernel (select the Kernel menu above --> Change Kernel).  Still have problems?  Try relaunching Jupyter Notebook from the terminal prompt.  Also, consult the forums for more troubleshooting tips.**  

## Import Packages

In [2]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
from helpers import *
%matplotlib inline

# Import jupyter widgets
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Ideas for Lane Detection Pipeline

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images  
`cv2.cvtColor()` to grayscale or change color  
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

## Test Images

Build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [3]:
import os
os.listdir("test_images/")

['solidYellowCurve.jpg',
 'solidYellowLeft.jpg',
 'challenge4.jpg',
 'challenge5.jpg',
 'challenge2.jpg',
 'challenge3.jpg',
 'challenge1.jpg',
 'solidYellowCurve2.jpg',
 'solidWhiteRight.jpg',
 'whiteCarLaneSwitch.jpg',
 'solidWhiteCurve.jpg']

## Build a Lane Finding Pipeline



Build the pipeline and run your solution on all test_images. Make copies into the `test_images_output` directory, and you can use the images in your writeup report.

Try tuning the various parameters, especially the low and high Canny thresholds as well as the Hough lines parameters.

In [4]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images_output directory.

In [5]:
# Function for image selection
def test_image(i):
    return mpimg.imread('test_images/' + os.listdir("test_images/")[i])

In [6]:
# Convert image to hsl space
def hls(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HLS)

# Convert image to hsv space
def hsv(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

In [7]:
# Function for thresholding color in RGB space
def color_threshold(img, red, green, blue):
    color_select = np.copy(img)
    thresholds = (img[:,:,0] < red) | (img[:,:,1] < green) | (img[:,:,2] < blue)
    color_select[thresholds] = [0,0,0]
    return color_select

In [8]:
# Binarize image
def binarize(img, threshold = 0):
    binary = np.copy(img)
    binary[binary > threshold] = 255
    return binary

#### Images in different color spaces

In [9]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1))
def colorspace_tune(img_n):
    img = test_image(img_n)
    
    f = plt.figure(figsize=(20, 4))
    sp = f.add_subplot(1, 3, 1)
    sp.imshow(hls(img))
    sp = f.add_subplot(1, 3, 2)
    sp.imshow(hsv(img))
    sp = f.add_subplot(1, 3, 3)
    sp.imshow(img)

interactive(children=(IntSlider(value=5, description='img_n', max=10), Output()), _dom_classes=('widget-intera…

Applying a mask on HLS space may be beneficial, since both the yellow and white lines are observed easily here.

#### Color selection (white): parameter tuning in HLS space

In [10]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1),
          h_low = (0, 255, 1),
          l_low = (0, 255, 1),
          s_low = (0, 255, 1),
          h_high = (0, 255, 1),
          l_high = (0, 255, 1),
          s_high = (0, 255, 1))
def hls_white_tune(img_n = 0, h_low = 0, l_low = 200, s_low = 0, h_high = 255, l_high = 255, s_high = 255):
    img = test_image(img_n)
    
    converted = hls(img)
    lower = np.uint8([h_low, l_low, s_low])
    upper = np.uint8([h_high, l_high, s_high])
    mask = cv2.inRange(converted, lower, upper)
    
    plt.imshow(cv2.bitwise_and(img, img, mask = mask))


interactive(children=(IntSlider(value=0, description='img_n', max=10), IntSlider(value=0, description='h_low',…

#### Color selection (yellow): parameter tuning in HLS space

In [11]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1),
          h_low = (0, 255, 1),
          l_low = (0, 255, 1),
          s_low = (0, 255, 1),
          h_high = (0, 255, 1),
          l_high = (0, 255, 1),
          s_high = (0, 255, 1))
def hls_yellow_tune(img_n = 0, h_low = 15, l_low = 0, s_low = 100, h_high = 35, l_high = 200, s_high = 255):
    img = test_image(img_n)
    
    converted = hls(img)
    lower = np.uint8([h_low, l_low, s_low])
    upper = np.uint8([h_high, l_high, s_high])
    mask = cv2.inRange(converted, lower, upper)
    
    plt.imshow(cv2.bitwise_and(img, img, mask = mask))


interactive(children=(IntSlider(value=0, description='img_n', max=10), IntSlider(value=15, description='h_low'…

#### Color selection: parameter tuning in RGB space

In [12]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1),
          red = (0, 255, 1),
          green = (0, 255, 1),
          blue = (0, 255, 1))
def color_tune(img_n = 0, red = 224, green = 136, blue = 0):
    img = test_image(img_n)
    
    f = plt.figure(figsize=(20, 4))
    sp = f.add_subplot(1, 2, 1)
    sp.imshow(color_threshold(img, red, green, blue))
    sp = f.add_subplot(1, 2, 2)
    sp.imshow(img)

interactive(children=(IntSlider(value=0, description='img_n', max=10), IntSlider(value=224, description='red',…

In [13]:
red = 224
green = 136
blue = 0

#### Canny: parameter tuning

In [14]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1),
          kernel_size = (1, 30, 2),
          low_threshold = (0, 255, 1),
          high_threshold = (0, 255, 1))
def canny_tune(img_n = 0, kernel_size = 3, low_threshold = 50, high_threshold = 150):
    img = test_image(img_n)
    
    f = plt.figure(figsize=(20, 4))
    sp = f.add_subplot(1, 2, 1)
    sp.imshow(canny(gaussian_blur(binarize(grayscale(color_threshold(img, red, green, blue))), kernel_size), low_threshold, high_threshold), cmap='gray')
    sp = f.add_subplot(1, 2, 2)
    sp.imshow(gaussian_blur(binarize(grayscale(color_threshold(img, red, green, blue))), kernel_size), cmap='gray')

interactive(children=(IntSlider(value=0, description='img_n', max=10), IntSlider(value=3, description='kernel_…

In [15]:
kernel_size = 7
low_threshold = 50
high_threshold = 150

#### ROI: parameter tuning

In [16]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1),
          center_base = (-0.5, 0.5, 0.01),
          center_top = (0, 100, 0.1),
          top = (0, 1.0, 0.01),
          width_base = (0, 1.0, 0.01),
          width_top = (0, 1.0, 0.01))
def roi_tuning(img_n = 0, center_base = 0.02, center_top = 0.5, top = 0.62, width_base = 0.84, width_top = 0.12):
    # Extract image data
    img = test_image(img_n)
    width = img.shape[1]
    height = img.shape[0]
    
    # Create vertices array
    vertices = np.array([[(width / 2 + center_base * width - (width_base * width) / 2, height),
                          (center_top  * width - (width_top * width)  / 2, height * top), 
                          (center_top  * width + (width_top * width)  / 2, height * top), 
                          (width / 2 + center_base * width + (width_base * width) / 2, height)]], dtype=np.int32)
    
    f = plt.figure(figsize=(20, 4))
    sp = f.add_subplot(1, 2, 1)
    sp.imshow(region_of_interest(canny(gaussian_blur(grayscale(color_threshold(img, red, green, blue)), kernel_size), low_threshold, high_threshold), vertices), cmap='gray')
    sp = f.add_subplot(1, 2, 2)
    sp.imshow(region_of_interest(img, vertices))

interactive(children=(IntSlider(value=0, description='img_n', max=10), FloatSlider(value=0.02, description='ce…

In [17]:
center_base = 0.02
center_top = 0.5
top = 0.62
width_base = 0.84
width_top = 0.12

#### Hough transform: parameter tuning

In [18]:
@interact(img_n = (0, len(os.listdir("test_images/")) - 1, 1),
          rho = (1, 100, 1),
          theta = (1, 180, 1), 
          threshold = (0, 100, 1), 
          min_line_len = (0, 200, 1), 
          max_line_gap = (0, 200, 1))
def hough_tuning(img_n = 0, rho = 2, theta = 1, threshold = 14, min_line_len = 60, max_line_gap = 60):
    img = test_image(img_n)
    width = img.shape[1]
    height = img.shape[0]
    
    # Create vertices array
    vertices = np.array([[(width / 2 + center_base * width - (width_base * width) / 2, height),
                          (center_top  * width - (width_top * width)  / 2, height * top), 
                          (center_top  * width + (width_top * width)  / 2, height * top), 
                          (width / 2 + center_base * width + (width_base * width) / 2, height)]], dtype=np.int32)
    
    f = plt.figure(figsize=(20, 4))
    img = region_of_interest(canny(gaussian_blur(grayscale(color_threshold(img, red, green, blue)), kernel_size), low_threshold, high_threshold), vertices)
    sp = f.add_subplot(1, 2, 1)
    sp.imshow(hough_lines(img, rho, theta * np.pi/180, threshold, min_line_len, max_line_gap), cmap='gray')
    sp = f.add_subplot(1, 2, 2)
    sp.imshow(img, cmap='gray')

interactive(children=(IntSlider(value=0, description='img_n', max=10), IntSlider(value=2, description='rho', m…

In [19]:
rho = 2
theta = 1
threshold = 20
min_line_len = 30
max_line_gap = 200

#### Pipeline

In [27]:
# Read image
image = test_image(0)
f = plt.figure(figsize=(20, 30))
sp = f.add_subplot(9, 1, 1)
sp.imshow(image)

# Threshold color
color = color_threshold(image, red, green, blue)
sp = f.add_subplot(9, 1, 2)
sp.imshow(color)

# Turn into grayscale
gray = grayscale(color)
sp = f.add_subplot(9, 1, 3)
sp.imshow(gray, cmap="gray")

# Binarize image
binary = binarize(gray)
sp = f.add_subplot(9, 1, 4)
sp.imshow(gray, cmap="gray")

# Apply gaussian filter
gaus = gaussian_blur(binary, kernel_size)
sp = f.add_subplot(9, 1, 5)
sp.imshow(gaus, cmap="gray")

# Apply Canny edge detector
edge = canny(gaus, low_threshold, high_threshold)
sp = f.add_subplot(9, 1, 6)
sp.imshow(edge, cmap="gray")

# Select region of interest
width = image.shape[1]
height = image.shape[0]

# Create vertices array
vertices = np.array([[(width / 2 + center_base * width - (width_base * width) / 2, height),
                      (center_top  * width - (width_top * width)  / 2, height * top), 
                      (center_top  * width + (width_top * width)  / 2, height * top), 
                      (width / 2 + center_base * width + (width_base * width) / 2, height)]], dtype=np.int32)

roi = region_of_interest(edge, vertices)
sp = f.add_subplot(9, 1, 7)
sp.imshow(roi, cmap="gray")

# Get Hough lines
hough = hough_lines(roi, rho, theta * np.pi/180, threshold, min_line_len, max_line_gap)
sp = f.add_subplot(9, 1, 8)
sp.imshow(hough, cmap="gray")

# Compute weighted image
weight = weighted_img(hough, image)
sp = f.add_subplot(9, 1, 9)
sp.imshow(weight)

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

**Note: if you get an import error when you run the next cell, try changing your kernel (select the Kernel menu above --> Change Kernel). Still have problems? Try relaunching Jupyter Notebook from the terminal prompt. Also, consult the forums for more troubleshooting tips.**

**If you get an error that looks like this:**
```
NeedDownloadError: Need ffmpeg exe. 
You can download it by calling: 
imageio.plugins.ffmpeg.download()
```
**Follow the instructions in the error message and check out [this forum post](https://discussions.udacity.com/t/project-error-of-test-on-videos/274082) for more troubleshooting tips across operating systems.**

In [22]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [23]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # Threshold color
    color = color_threshold(image, red, green, blue)
    
    # Turn into grayscale
    gray = grayscale(color)
    
    # Binarize image
    binary = binarize(gray)

    # Apply gaussian filter
    gaus = gaussian_blur(binary, kernel_size)

    # Apply Canny edge detector
    edge = canny(gaus, low_threshold, high_threshold)
    
    # Select region of interest
    width = image.shape[1]
    height = image.shape[0]
    vertices = np.array([[(width / 2 + center_base * width - (width_base * width) / 2, height),
                          (center_top  * width - (width_top * width)  / 2, height * top), 
                          (center_top  * width + (width_top * width)  / 2, height * top), 
                          (width / 2 + center_base * width + (width_base * width) / 2, height)]], dtype=np.int32)
    roi = region_of_interest(edge, vertices)

    # Get Hough lines
    hough = hough_lines(roi, rho, theta * np.pi/180, threshold, min_line_len, max_line_gap)

    # Compute weighted image
    result = weighted_img(hough, image)
    
    return result

Let's try the one with the solid white lane on the right first ...

In [24]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|▏         | 5/221 [00:00<00:04, 45.95it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
CPU times: user 5.35 s, sys: 1.63 s, total: 6.98 s
Wall time: 6.01 s


Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [25]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

## Improve the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**Go back and modify your draw_lines function accordingly and try re-running your pipeline. The new output should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [26]:
# New Helper Functions

# Calculate slope of a line
def slope(line):
    for x1,y1,x2,y2 in line:
        # Calculate the slope (as if the bottom left corner were (0,0))
        # (ImageHeight - y2) - (ImageHeight - y1) = (y1 - y2) 
        return (y2-y1)/(x2-x1) 

def length(line):
    for x1,y1,x2,y2 in line:
        return math.sqrt((y2-y1)**2 + (x2-x1)**2)
    
def draw_lines(img, lines, color=[0, 255, 0], thickness=5):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    
    if lines is None:
        return img
    
    
    # Group by slope's sign (inverted because of image's axis)
    right = [line for line in lines if slope(line) > 0]
    left = [line for line in lines if slope(line) < 0]    
    
    # Get average slope and intercept for right
    slopes = list(map(slope, right))
    points = [[line[0][0], line[0][1]] for line in right]
    lengths = list(map(length, right))
    intercepts = [point[1] - slope * point[0] for point, slope in zip(points, slopes)]
    
    m = sum([ slope * length for slope, length in zip(slopes, lengths)]) / sum(lengths)
    b = sum([ intercept * length for intercept, length in zip(intercepts, lengths)]) / sum(lengths)
    
    y1 = img.shape[0]
    y2 = img.shape[0] * top
    x1 = (y1 - b)/m
    x2 = (y2 - b)/m
    
    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)
    
    # Get average slope and intercept for left
    slopes = list(map(slope, left))
    points = [[line[0][0], line[0][1]] for line in left]
    lengths = list(map(length, left))
    intercepts = [point[1] - slope * point[0] for point, slope in zip(points, slopes)]
    
    m = sum([ slope * length for slope, length in zip(slopes, lengths)]) / sum(lengths)
    b = sum([ intercept * length for intercept, length in zip(intercepts, lengths)]) / sum(lengths)
    
    y1 = img.shape[0]
    y2 = img.shape[0] * top
    x1 = (y1 - b)/m
    x2 = (y2 - b)/m
    
    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), color, thickness)

    return img

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

In [25]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   1%|          | 6/681 [00:00<00:12, 53.14it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
CPU times: user 15.5 s, sys: 4.36 s, total: 19.8 s
Wall time: 16.4 s


In [26]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Writeup and Submission

If you're satisfied with your video outputs, it's time to make the report writeup in a pdf or markdown file. Once you have this Ipython notebook ready along with the writeup, it's time to submit for review! Here is a [link](https://github.com/udacity/CarND-LaneLines-P1/blob/master/writeup_template.md) to the writeup template file.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [27]:
challenge_output = 'test_videos_output/challenge.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
##clip3 = VideoFileClip('test_videos/challenge.mp4').subclip(0,5)
clip3 = VideoFileClip('test_videos/challenge.mp4')
challenge_clip = clip3.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

t:   2%|▏         | 4/251 [00:00<00:08, 29.15it/s, now=None]

Moviepy - Building video test_videos_output/challenge.mp4.
Moviepy - Writing video test_videos_output/challenge.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/challenge.mp4
CPU times: user 10 s, sys: 3.16 s, total: 13.2 s
Wall time: 11.2 s


In [28]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))